In [ ]:
######### Package Imports #########################################################################
import os, warnings, copy, torch, ax
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
# from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from botorch.acquisition.multi_objective.logei import qLogExpectedHypervolumeImprovement
from ax.modelbridge.transforms.standardize_y import StandardizeY
from ax.modelbridge.transforms.unit_x import UnitX
from ax.modelbridge.transforms.remove_fixed import RemoveFixed
from ax.modelbridge.transforms.log import Log
from ax.core.base_trial import TrialStatus as T
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.plot.slice import plot_slice

from optimpv import *
from optimpv.axBOtorch.axBOtorchOptimizer import axBOtorchOptimizer
from optimpv.DDfits.HysteresisAgent import HysteresisAgent

import pySIMsalabim as sim
from pySIMsalabim.experiments.JV_steady_state import *
from pySIMsalabim.experiments.hysteresis import *

init_notebook_plotting()
warnings.filterwarnings('ignore') 
##############################################################################################
# Define the parameters to be fitted
params = []

mun = FitParam(name = 'l2.mu_n', value = 8e-5, bounds = [1e-5,1e-2], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_n$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mun)

mup = FitParam(name = 'l2.mu_p', value = 8e-5, bounds = [1e-5,1e-2], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_p$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(mup)

bulk_tr = FitParam(name = 'l2.N_t_bulk', value = 5.1e20, bounds = [1e19,1e22], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{T}$', unit='s', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(bulk_tr)

Nions = FitParam(name = 'l2.N_ions', value = 4e22, bounds = [1e22,5e22], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{ions}$', unit='m$^{-3}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(Nions)


muions = FitParam(name = 'l2.mu_ions', value = 5.7e-12, bounds = [1e-13,1e-11], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-12, rescale = True, stepsize = None, display_name=r'$\mu_{ions}$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(muions)

# mu_cation = FitParam(name = 'l2.mu_cation', value = 1e-11, bounds = [1e-13,1e-11], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-12, rescale = True, stepsize = None, display_name=r'$\mu_{cation}$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
# params.append(mu_cation)

# mu_anion = FitParam(name = 'l2.mu_anion', value = 5.7e-12, bounds = [1e-12,1e-11], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-12, rescale = True, stepsize = None, display_name=r'$\mu_{anion}$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
# params.append(mu_anion)

# muions_l1 = FitParam(name = 'l1.mu_ions', value = 5.7e-15, bounds = [1e-20,1e-14], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-12, rescale = True, stepsize = None, display_name=r'$\mu_{ions1}$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
# params.append(muions_l1)

# muions_l3 = FitParam(name = 'l3.mu_ions', value = 5.7e-15, bounds = [1e-16,1e-11], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-12, rescale = True, stepsize = None, display_name=r'$\mu_{ions3}$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
# params.append(muions_l3)

N_t_int_l1 = FitParam(name = 'l1.N_t_int', value = 5e12, bounds = [5e11,5e13], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{T,int}^{HTL}$', unit='m$^{-2}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(N_t_int_l1)

N_t_int_l2 = FitParam(name = 'l2.N_t_int', value = 4e12, bounds = [5e11,5e13], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{T,int}^{ETL}$', unit='m$^{-2}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
params.append(N_t_int_l2)

eps_r = FitParam(name = 'l2.eps_r', value = 24, bounds = [20,40], type='range', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\epsilon_r$', unit='-', axis_type = 'linear', std = 0,encoding = None,force_log = False)
params.append(eps_r)



# eps_rl1 = FitParam(name = 'l1.eps_r', value = 3.5, bounds = [2.9,4], type='range', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\epsilon_{r}^{HTL}$', unit='-', axis_type = 'linear', std = 0,encoding = None,force_log = False)
# params.append(eps_rl1)

# eps_rl3 = FitParam(name = 'l3.eps_r', value = 3.5, bounds = [3.5,5], type='range', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\epsilon_{r}^{ETL}$', unit='-', axis_type = 'linear', std = 0,encoding = None,force_log = False)
# params.append(eps_rl3)

# mu_pl1 = FitParam(name = 'l1.mu_p', value = 6e-8, bounds = [1e-9,1e-7], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-8, rescale = True, stepsize = None, display_name=r'$\mu_{p}^{HTL}$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
# params.append(mu_pl1)

# mu_nl3 = FitParam(name = 'l3.mu_n', value = 1e-6, bounds = [1e-7,5e-6], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = 1e-8, rescale = True, stepsize = None, display_name=r'$\mu_{n}^{ETL}$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)
# params.append(mu_nl3)

# Ev_l1 = FitParam(name = 'l1.Ev', value = 5.2, bounds = [5.1,5.3], values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$E_{V}^{HTL}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)
# params.append(Ev_l1)

# offset_W_L = FitParam(name = 'offset_W_L.E_v', value = 0.05, bounds = [0,0.2], type='fixed', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta W_L$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)
# params.append(offset_W_L)

#original values
params_orig = copy.deepcopy(params)


In [ ]:
path2data = os.path.join('/home/lecorre/Desktop/hysteresis_fits_Koster/', 'PTAA_fresh')
scan_speeds = [0.23, 3, 496, 1780]

# Define the experiment
scan_speed1 = 0.23
df = pd.read_csv(os.path.join(path2data, str(scan_speed1)+'_JV_BSpline_mod_bwd.txt'), sep=r'\s+')
X1 = np.asarray(df['Vext'].values)
y1 = np.asarray(df['Jext'].values)
df = pd.read_csv(os.path.join(path2data, str(scan_speed1)+'_JV_BSpline_mod_fwd.txt'), sep=r'\s+')
X1 = np.concatenate((X1, np.asarray(df['Vext'].values)))
y1 = np.concatenate((y1, np.asarray(df['Jext'].values)))
Vmin1 = np.min(X1)
Vmax1 = np.max(X1)

scan_speed2 = 0.83
df = pd.read_csv(os.path.join(path2data, str(scan_speed2)+'_JV_BSpline_mod_bwd.txt'), sep=r'\s+')
X2 = np.asarray(df['Vext'].values)
y2 = np.asarray(df['Jext'].values)
df = pd.read_csv(os.path.join(path2data, str(scan_speed2)+'_JV_BSpline_mod_fwd.txt'), sep=r'\s+')
X2 = np.concatenate((X2, np.asarray(df['Vext'].values)))
y2 = np.concatenate((y2, np.asarray(df['Jext'].values)))
Vmin2 = np.min(X2)
Vmax2 = np.max(X2)

scan_speed3 = 10.7
df = pd.read_csv(os.path.join(path2data, str(scan_speed3)+'_JV_BSpline_mod_bwd.txt'), sep=r'\s+')
X3 = np.asarray(df['Vext'].values)
y3 = np.asarray(df['Jext'].values)
df = pd.read_csv(os.path.join(path2data, str(scan_speed3)+'_JV_BSpline_mod_fwd.txt'), sep=r'\s+')
X3 = np.concatenate((X3, np.asarray(df['Vext'].values)))
y3 = np.concatenate((y3, np.asarray(df['Jext'].values)))
Vmin3 = np.min(X3)
Vmax3 = np.max(X3)

scan_speed4 = 138
df = pd.read_csv(os.path.join(path2data, str(scan_speed4)+'_JV_BSpline_mod_bwd.txt'), sep=r'\s+')
X4 = np.asarray(df['Vext'].values)
y4 = np.asarray(df['Jext'].values)
df = pd.read_csv(os.path.join(path2data, str(scan_speed4)+'_JV_BSpline_mod_fwd.txt'), sep=r'\s+')
X4 = np.concatenate((X4, np.asarray(df['Vext'].values)))
y4 = np.concatenate((y4, np.asarray(df['Jext'].values)))
Vmin4 = np.min(X4)
Vmax4 = np.max(X4)

plt.figure()
viridis = plt.cm.get_cmap('viridis', 4)
plt.plot(-X1, -y1, color=viridis(0))
plt.plot(-X2, -y2, color=viridis(1))
plt.plot(-X3, -y3, color=viridis(2))
plt.plot(-X4, -y4, color=viridis(3))

plt.ylim(-220,0)
plt.xlabel('Applied Voltage [V]')
plt.ylabel('Current Density [A/m$^2$]')
plt.show()




In [ ]:
# session_path_simss = os.path.join('/home/lecorre/Desktop/pySIMsalabim/', 'SIMsalabim','SimSS')
# simss_device_parameters = os.path.join(session_path_simss, 'simulation_setup.txt')
session_path_zimt = os.path.join('/home/lecorre/Desktop/hysteresis_fits_Koster/', 'SIMsalabim','ZimT')
zimt_device_parameters = os.path.join(session_path_zimt, 'simulation_setup.txt')

# Set the JV parameters
direction = 1
G_frac = 1.14
Vmin = -1.1
Vmax = 0
steps = 200
Gfracs = None

cmd_pars = []
for param in params:
    if param.name != 'l2.C_np_bulk' and param.name != 'offset_l2_l1.E_c' and param.name != 'offset_l2_l3.E_v' and param.name != 'Egap_l1.E_v' and param.name != 'offset_W_L.E_c' and param.name != 'l2.N_ions' and param.name != 'l2.mu_ions' and param.name != 'l1.mu_ions' and param.name != 'l3.mu_ions':
        cmd_pars.append({'par':param.name, 'val':str(param.value)})
    elif param.name == 'offset_l2_l1.E_c':
        cmd_pars.append({'par':'l1.E_c', 'val':str(3.9-param.value)})
        vv = 3.9-param.value
  
    elif param.name == 'l2.C_np_bulk':
        cmd_pars.append({'par':'l2.C_n_bulk', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.C_p_bulk', 'val':str(param.value)})

    elif param.name == 'offset_l2_l3.E_v':
        cmd_pars.append({'par':'l3.E_v', 'val':str(5.53-param.value)})
    
    elif param.name == 'Egap_l1.E_v':
        cmd_pars.append({'par':'l1.E_v', 'val': str(vv+param.value)})
    
    elif param.name == 'offset_W_L.E_c':
        cmd_pars.append({'par':'W_L', 'val':str(vv-param.value)})

    elif param.name == 'l2.N_ions':
        cmd_pars.append({'par':'l2.N_cation', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.N_anion', 'val':str(param.value)})

    elif param.name == 'l2.mu_ions':
        cmd_pars.append({'par':'l2.mu_cation', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.mu_anion', 'val':str(param.value)})

    elif param.name == 'l1.mu_ions':
        cmd_pars.append({'par':'l1.mu_cation', 'val':str(param.value)})
        cmd_pars.append({'par':'l1.mu_anion', 'val':str(param.value)})

    elif param.name == 'l3.mu_ions':
        cmd_pars.append({'par':'l3.mu_cation', 'val':str(param.value)})
        cmd_pars.append({'par':'l3.mu_anion', 'val':str(param.value)})



In [ ]:
# t_ = time.time()
# # scan_speed1 = 0.23
# UUID = str(uuid.uuid4())
# # Run the JV simulation
# ret, mess, rms = Hysteresis_JV(zimt_device_parameters, session_path_zimt, 0, scan_speed1, direction, G_frac, Vmin=Vmin1, Vmax=Vmax1, steps = steps, UUID = UUID, cmd_pars = cmd_pars)
# print('scan_speed1 done in', time.time()-t_)
# # Load the data
# data = pd.read_csv(os.path.join(session_path_zimt, 'tj_'+UUID+'.dat'), sep=r'\s+')

# # Extract the data
# X1fit = np.asarray(data['Vext'].values)
# y1fit = np.asarray(data['Jext'].values)

# t_ = time.time()
# # scan_speed2 = 0.83
# UUID = str(uuid.uuid4())
# # Run the JV simulation
# ret, mess, rms = Hysteresis_JV(zimt_device_parameters, session_path_zimt, 0, scan_speed2, direction, G_frac, Vmin=Vmin2, Vmax=Vmax2, steps = steps, UUID = UUID, cmd_pars = cmd_pars)
# print('scan_speed2 done in', time.time()-t_)
# # Load the data
# data = pd.read_csv(os.path.join(session_path_zimt, 'tj_'+UUID+'.dat'), sep=r'\s+')

# # Extract the data
# X2fit = np.asarray(data['Vext'].values)
# y2fit = np.asarray(data['Jext'].values)

# t_ = time.time()
# # scan_speed3 = 496
# UUID = str(uuid.uuid4())
# # Run the JV simulation
# ret, mess, rms = Hysteresis_JV(zimt_device_parameters, session_path_zimt, 0, scan_speed3, direction, G_frac, Vmin=Vmin3, Vmax=Vmax3, steps = steps, UUID = UUID, cmd_pars = cmd_pars)
# print('scan_speed3 done in', time.time()-t_)
# # Load the data
# data = pd.read_csv(os.path.join(session_path_zimt, 'tj_'+UUID+'.dat'), sep=r'\s+')

# # Extract the data
# X3fit = np.asarray(data['Vext'].values)
# y3fit = np.asarray(data['Jext'].values)

# t_ = time.time()
# # scan_speed4 = 6400
# UUID = str(uuid.uuid4())
# # Run the JV simulation
# ret, mess, rms = Hysteresis_JV(zimt_device_parameters, session_path_zimt, 0, scan_speed4, direction, G_frac, Vmin=Vmin4, Vmax=Vmax4, steps = steps, UUID = UUID, cmd_pars = cmd_pars)
# print('scan_speed4 done in', time.time()-t_)
# # Load the data
# data = pd.read_csv(os.path.join(session_path_zimt, 'tj_'+UUID+'.dat'), sep=r'\s+')

# # Extract the data
# X4fit = np.asarray(data['Vext'].values)
# y4fit = np.asarray(data['Jext'].values)


In [ ]:
# plt.figure(figsize=(20,8))
# # 2x2 plot
# ax1 = plt.subplot(1,4,1)
# plt.plot(-X1,-y1)
# plt.plot(-X1fit,-y1fit, 'r--')
# plt.title('Scan speed = {} V/s'.format(scan_speed1))
# plt.xlabel('Applied Voltage [V]')
# plt.ylabel('Current Density [A/m$^2$]')
# # plt.ylim(-220,50)

# ax2 = plt.subplot(1,4,2)
# plt.plot(-X2,-y2)
# plt.plot(-X2fit,-y2fit, 'r--')
# plt.title('Scan speed = {} V/s'.format(scan_speed2))
# plt.xlabel('Applied Voltage [V]')
# plt.ylabel('Current Density [A/m$^2$]')
# # plt.ylim(-220,50)

# ax3 = plt.subplot(1,4,3)
# plt.plot(-X3,-y3)
# plt.plot(-X3fit,-y3fit, 'r--')
# plt.title('Scan speed = {} V/s'.format(scan_speed3))
# plt.xlabel('Applied Voltage [V]')
# plt.ylabel('Current Density [A/m$^2$]')
# # plt.ylim(-220,50)

# ax4 = plt.subplot(1,4,4)
# plt.plot(-X4,-y4)
# plt.plot(-X4fit,-y4fit, 'r--')
# plt.title('Scan speed = {} V/s'.format(scan_speed4))
# plt.xlabel('Applied Voltage [V]')
# plt.ylabel('Current Density [A/m$^2$]')
# # plt.ylim(-220,50)

# print(min(-y1), min(-y1fit))
# print(min(-y4), min(-y4fit))

In [ ]:
# Define the agents
session_path_zimt = os.path.join('/home/lecorre/Desktop/hysteresis_fits_Koster/', 'SIMsalabim','ZimT')
zimt_device_parameters = os.path.join(session_path_zimt, 'simulation_setup.txt')

# Set the JV parameters
direction = 1
G_frac = 1.14
# Vmin = -1.1
# Vmax = 0
steps = 200
metric = 'nrmse'
loss = 'log10'
threshold = -1

hyst1 = HysteresisAgent(params, X1, y1, session_path_zimt, Vmin1, Vmax1, scan_speed1, steps, direction, G_frac=G_frac, simulation_setup=zimt_device_parameters, metric=[metric], loss=[loss], threshold= [threshold],minimize=[True], name='Hyst1')

hyst2 = HysteresisAgent(params, X2, y2, session_path_zimt, Vmin2, Vmax2, scan_speed2, steps, direction, G_frac=G_frac, simulation_setup=zimt_device_parameters, metric=[metric], loss=[loss], threshold= [threshold],minimize=[True], name='Hyst2')

hyst3 = HysteresisAgent(params, X3, y3, session_path_zimt, Vmin3, Vmax3, scan_speed3, steps, direction, G_frac=G_frac, simulation_setup=zimt_device_parameters, metric=[metric], loss=[loss], threshold= [threshold],minimize=[True], name='Hyst3')

hyst4 = HysteresisAgent(params, X4, y4, session_path_zimt, Vmin4, Vmax4, scan_speed4, steps, direction, G_frac=G_frac, simulation_setup=zimt_device_parameters, metric=[metric], loss=[loss], threshold= [threshold],minimize=[True], name='Hyst4')

In [ ]:
# dum_dic = {}
# for param in params:
#     dum_dic[param.name] = param.value

# print(hyst1.run_Ax(dum_dic), hyst2.run_Ax(dum_dic), hyst3.run_Ax(dum_dic), hyst4.run_Ax(dum_dic))

In [ ]:
# Define the model kwargs
model_kwargs_list = [{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogExpectedHypervolumeImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY]}]
parameter_constraints = [f'l2.mu_cation - l1.mu_ions >= {0}',f'l2.mu_cation - l3.mu_ions >= {0}',f'l2.mu_anion - l1.mu_ions >= {0}',f'l2.mu_anion - l3.mu_ions >= {0}']
parameter_constraints = None

# Define the optimizer
optimizer = axBOtorchOptimizer(params = params, agents = [hyst1, hyst2, hyst3, hyst4 ], models = ['SOBOL','BOTORCH_MODULAR'],n_batches = [1,30], batch_size = [10,2], ax_client = None,  max_parallelism = -1, model_kwargs_list = model_kwargs_list, model_gen_kwargs_list = None, name = 'ax_opti', parallel_agents = True, parameter_constraints = parameter_constraints)



In [ ]:
optimizer.optimize()

In [ ]:
ax_client = optimizer.ax_client
pareto = ax_client.get_pareto_optimal_parameters(use_model_predictions=False)
best_keys = list(pareto.keys())
print(pareto)
plt.figure()
plt.plot(ax_client.get_trace())

#find max in get_trace
max_val = np.max(ax_client.get_trace())
index_max = np.argmax(ax_client.get_trace())
index_max = best_keys[0]
best_parameters = pareto[index_max][0]

hyst1.params_w(best_parameters,hyst1.params)
print(hyst1.get_SIMsalabim_clean_cmd(hyst1.params))
hyst1.package_SIMsalabim_files(hyst1.params,'zimt')

In [ ]:
# plot the evolution of the optimization
render(ax_client.get_contour_plot(param_x="l2.mu_n", param_y="l2.mu_p", metric_name=optimizer.all_metrics[0]))

model = ax_client.generation_strategy.model

render(plot_slice(model=model, param_name="l2.mu_n", metric_name=optimizer.all_metrics[0]))
render(plot_slice(model=model, param_name="l2.mu_p", metric_name=optimizer.all_metrics[0]))
# render(plot_slice(model=model, param_name="l2.N_ions", metric_name=optimizer.all_metrics[0]))
# render(plot_slice(model=model, param_name="l2.C_np_bulk", metric_name=metric))


In [ ]:
data = ax_client.experiment.fetch_data()
# split df by metric name 
data = data.df
metric1_df = data[data["metric_name"] == optimizer.all_metrics[0]]
metric2_df = data[data["metric_name"] == optimizer.all_metrics[1]]
metric3_df = data[data["metric_name"] == optimizer.all_metrics[2]]
metric4_df = data[data["metric_name"] == optimizer.all_metrics[3]]

# reset index
metric1_df = metric1_df.reset_index(drop=True)
metric2_df = metric2_df.reset_index(drop=True)
metric3_df = metric3_df.reset_index(drop=True)
metric4_df = metric4_df.reset_index(drop=True)
# plt.figure()
# plt.plot(np.minimum.accumulate(data.df["mean"]), label="Best value seen so far")

plt.figure()
plt.plot(np.minimum.accumulate(metric1_df["mean"]), label="Best value seen so far")
plt.xlabel('Number of iterations')
plt.ylabel('log of '+optimizer.all_metrics[0])

plt.figure()
plt.plot(np.minimum.accumulate(metric2_df["mean"]), label="Best value seen so far")
plt.xlabel('Number of iterations')
plt.ylabel('log of '+optimizer.all_metrics[1])

plt.figure()
plt.plot(np.minimum.accumulate(metric3_df["mean"]), label="Best value seen so far")
plt.xlabel('Number of iterations')
plt.ylabel('log of '+optimizer.all_metrics[2])

plt.figure()
plt.plot(np.minimum.accumulate(metric4_df["mean"]), label="Best value seen so far")
plt.xlabel('Number of iterations')
plt.ylabel('log of '+optimizer.all_metrics[3])


plt.show()

In [ ]:
# create dic with keys same as ax_client.experiment.trials[0].arm.parameters
dumdic = {}
for key in ax_client.experiment.trials[0].arm.parameters.keys():
    dumdic[key] = []

# fill the dic with the values of the parameters
for i in range(len(ax_client.experiment.trials)):
    if ax_client.experiment.trials[i].status == T.COMPLETED:
        for key in ax_client.experiment.trials[i].arm.parameters.keys():
            dumdic[key].append(ax_client.experiment.trials[i].arm.parameters[key])


data = ax_client.experiment.fetch_data().df

target1 = data[data['metric_name'] == optimizer.all_metrics[0]]['mean']

dumdic[optimizer.all_metrics[0]] = list(target1)

target2 = data[data['metric_name'] ==  optimizer.all_metrics[1]]['mean']

dumdic[optimizer.all_metrics[1]] = list(target2)

target3 = data[data['metric_name'] ==  optimizer.all_metrics[2]]['mean']

dumdic[optimizer.all_metrics[2]] = list(target3)

target4 = data[data['metric_name'] ==  optimizer.all_metrics[3]]['mean']

dumdic[optimizer.all_metrics[3]] = list(target4)

dumdic['iteration'] = list(data[data['metric_name'] == optimizer.all_metrics[0]]['trial_index'])

df = pd.DataFrame(dumdic)


for par in params:
    if par.name in df.columns:
        if par.rescale:
            df[par.name] = df[par.name] *par.fscale


In [ ]:
# get name of all parameters that are not 'fixed'
names = []
display_names = []
log_scale = []
axis_limits = []
for p in params:
    if p.type != 'fixed':
        names.append(p.name)
        display_names.append(p.display_name + ' [' + p.unit + ']')
        log_scale.append(p.axis_type == 'log')
        axis_limits.append(p.bounds)

# Get all combinations of names
comb = list(combinations(names, 2))

# Determine the grid size
n = len(names) 

fig, axes = plt.subplots(n, n, figsize=(25, 25))

# Plot each combination in the grid
for i, xx in enumerate(names):
    for j, yy in enumerate(names):
        xval = np.nan
        yval = np.nan

        for p in params_orig:
            if p.name == xx:
                xval = p.value
            if p.name == yy:
                yval = p.value

        ax = axes[i, j]
        
        if i == j:
            # kde plot on the diagonal
            sns.kdeplot(x=yy, data=df, ax=ax, fill=True, thresh=0, levels=100, cmap="rocket", color="#03051A", log_scale=log_scale[names.index(xx)])

            ax.axvline(x=yval, color='r', linestyle='-')
            # put point at the best value top of the axis
           

            if log_scale[names.index(yy)]:
                ax.set_xscale('log')
                ax.set_xlim(axis_limits[names.index(yy)])
            else:
                ax.set_xlim(axis_limits[names.index(yy)])
            
            # put x label on the top
            # except for the last one
            if i < n - 1:
                ax.xaxis.set_label_position('top')
                ax.xaxis.tick_top()

        elif i > j:
            kind = 'kde'
            if kind == 'scatter':
                sns.scatterplot(x=yy, y=xx, data=df, ax=ax, color="#03051A")
                ax.set_xscale('log')
                ax.set_yscale('log')
            else:
                sns.kdeplot(x=yy, y=xx, data=df, ax=ax, fill=True, thresh=0, levels=100, cmap="rocket", color="#03051A", log_scale=(log_scale[names.index(yy)], log_scale[names.index(xx)]))

            xval = np.nan
            yval = np.nan
            for p in params_orig:
                if p.name == xx:
                    xval = p.value
                elif p.name == yy:
                    yval = p.value

            # Plot as line over the full axis
            ax.axhline(y=xval, color='r', linestyle='-')
            ax.axvline(x=yval, color='r', linestyle='-')
            ax.scatter(best_parameters[yy], best_parameters[xx], c='y', marker='*', s=200, zorder=10)
            
            ax.set_xlim(axis_limits[names.index(yy)])
            ax.set_ylim(axis_limits[names.index(xx)])
        else:
            ax.set_visible(False)


        if j == 0:
            ax.set_ylabel(display_names[i])
        else:
            ax.tick_params(axis='y', which='both', left=True, labelleft=False)
            ax.set_ylabel('')

        if i == n - 1:
            ax.set_xlabel(display_names[j])
            # rotate x labels
            ax.tick_params(axis='x', rotation=45, which='both')
        else:
            ax.tick_params(axis='x', which='both', bottom=True, labelbottom=False)
            ax.set_xlabel('')

        # if i==0 and j==0:
        #     ax.tick_params(axis='y', which='both', left=True, labelleft=False)
        #     ax.set_ylabel('')

        if i == j:
            # xlabel at the top
            ax.tick_params(axis='x', which='both', top=True, labeltop=False)
            # ax.xaxis.set_label_position('top')
            # ax.set_xlabel(display_names[j])
            ax.set_xlabel('')


        # Optionally, set labels for the last row and first column
        # else:
        #     if i == 0:
        #         ax.set_xlabel(yy)
        #     if j == 0:
        #         ax.set_ylabel(xx)
        


plt.tight_layout()
plt.show()

In [ ]:
# print(best_parameters)
# print(hyst1.run_Ax(best_parameters), hyst2.run_Ax(best_parameters), hyst3.run_Ax(best_parameters), hyst4.run_Ax(best_parameters)
# best_parameters['l2.N_ions'] = 4e22

In [ ]:
# rerun the simulation with the best parameters
plt.figure(figsize=(20,10))
viridis = plt.cm.get_cmap('viridis', 4)
# 2x2 plot
ax1 = plt.subplot(1,4,1)
plt.plot(-hyst1.X[0],-hyst1.y[0],label='data',color=viridis(0))
plt.plot(-hyst1.X[0],-hyst1.run(parameters=best_parameters,exp_format='JV'),label='fit',linestyle='--',color=viridis(0))

ax2 = plt.subplot(1,4,2)
plt.plot(-hyst2.X[0],-hyst2.y[0],label='data',color=viridis(1))
plt.plot(-hyst2.X[0],-hyst2.run(parameters=best_parameters,exp_format='JV'),label='fit',linestyle='--',color=viridis(1))

ax3 = plt.subplot(1,4,3)
plt.plot(-hyst3.X[0],-hyst3.y[0],label='data',color=viridis(2))
plt.plot(-hyst3.X[0],-hyst3.run(parameters=best_parameters,exp_format='JV'),label='fit',linestyle='--',color=viridis(2))

ax4 = plt.subplot(1,4,4)
plt.plot(-hyst4.X[0],-hyst4.y[0],label='data',color=viridis(3))
plt.plot(-hyst4.X[0],-hyst4.run(parameters=best_parameters,exp_format='JV'),label='fit',linestyle='--',color=viridis(3))

plt.xlabel('Applied Voltage [V]')
plt.ylabel('Current Density [A/m$^2$]')
plt.ylim(-220,50)
plt.legend()
plt.show()

print(min(-hyst1.y[0]), min(-hyst1.run(parameters=best_parameters,exp_format='JV')))
print(min(-hyst4.y[0]), min(-hyst4.run(parameters=best_parameters,exp_format='JV')))

In [ ]:
# Clean up the output files (comment out if you want to keep the output files)
sim.clean_all_output(session_path_zimt)
sim.delete_folders('tmp',session_path_zimt)